### notebook to find the amount i spent at Amazon. Amazon said I could save 5K with their 5% cashback card. I wanted to confirm it. 
### websites said one can get reports from accounts tab but I didn't find any. so wrote this makeshift code to find out what I spent

In [ ]:
import os
os.chdir('C:\\Users\\anubr\\jupyter_notebooks\\web_scrape')

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'all_text',
 'amazon_orders.txt',
 'amazon_spend.ipynb',
 'amazon_spend_2.ipynb',
 'data',
 'How to Extract Data from PDF Forms Using Python _ by ankur garg _ Towards Data Science.pdf',
 'orders_placed.txt',
 'pdf2txt.py',
 'test.txt',
 'text.txt']

In [3]:
from io import StringIO
import io
import sys

In [4]:
import pdfminer

In [5]:
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

In [7]:
import re

In [8]:
lst_file=os.listdir(os.getcwd()) # list of files in folder

In [9]:
pdf=lambda x:re.search('.pdf$',x) # lambda function to search for pdf files 

In [10]:
pdf_file=list(filter(pdf,lst_file)) # filter only pdf files in the list

In [11]:
pdf_file

['page1.pdf',
 'page10.pdf',
 'page11.pdf',
 'page12.pdf',
 'page13.pdf',
 'page14.pdf',
 'page15.pdf',
 'page16.pdf',
 'page17.pdf',
 'page2.pdf',
 'page3.pdf',
 'page4.pdf',
 'page5.pdf',
 'page6.pdf',
 'page7.pdf',
 'page8.pdf',
 'page9.pdf']

In [12]:
output_string=StringIO()

In [50]:
# function to convert pdf to text with pdfminer
def pdf_to_txt(pdf_list):    
    for file in enumerate(pdf_file):
        infile=open(file[1],'rb')
        outfile=open(str(file[1])[:-4]+'_output.txt','w')
        parser=PDFParser(infile)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)

        outfile.write(output_string.getvalue())
        infile.close()
        outfile.close()    

In [58]:
# function to join output of individual pdf files processed
def join_files():   
    lst=[]
    pattern=re.compile('output.txt$')
    for item in os.listdir():
        if re.search(pattern,item):
            lst.append(item)
    outfile=open ('all_text','a')
    for file in lst:
        infile=open(file,'r')
        data=infile.read()
        outfile.write(data)
        infile.close()
    outfile.close()        

In [59]:
join_files()

In [6]:
# function to compile the orders placed
def spend_list(string=False,file=False):    
    with open (file,'r') as txtinfile:
        datum=txtinfile.read()
    spnd_lst=[]
    for inp in re.finditer(string,datum):
        strt=inp.start()
        new_txt=datum[strt:strt+100]
        new_lst=new_txt.split('\n')
        for item in new_lst:
            if 'PLACED' in item:
                y=new_txt.index(item)        
                date=str(new_txt[y:y+30].split('\n')[1])        

            if 'TOTAL' in item:
                n=new_txt.index(item)
                amount=new_txt[n:n+20].split('\n')[1].replace(u'\xa0','').encode('utf-8')
                amount=amount.decode('utf-8')
                if ',' in amount:
                    inp1=amount.split(',')
                    amount="".join(inp1)
                #namount=float(amount)    
        inp_val=(date,amount)
        spnd_lst.append(inp_val)
    return spnd_lst

In [7]:
lst1=spend_list('ORDER PLACED','all_text')

In [8]:
import numpy as np

In [9]:
import  pandas as pd
pd.set_option('display.max_rows',None)
lst_arr=np.asarray(lst1)
lst_df=pd.DataFrame(lst_arr)
lst_df.columns=['date','amount']

In [10]:
lst_df.head()

,date,amount
0,3 December 2020,299.00
1,3 December 2020,699.00
2,29 November 2020,900.00
3,28 November 2020,399.00
4,28 November 2020,1170.00


In [11]:
df_col_type=lst_df.dtypes.reset_index()
df_col_type.columns=['type','count']
df_col_type

,type,count
0,date,object
1,amount,object


In [12]:
# convert date to date and amount to float
lst_df['date']=lst_df['date'].astype('datetime64[ns]')
lst_df['amount']=pd.to_numeric(lst_df['amount'],downcast='float')

In [13]:
lst_df.head()

,date,amount
0,2020-12-03,299.0
1,2020-12-03,699.0
2,2020-11-29,900.0
3,2020-11-28,399.0
4,2020-11-28,1170.0


In [14]:
lst_df.shape

(1521, 2)

In [19]:
lst_df.sort_values('date').head()

,date,amount
1440,2020-01-03,265.0
864,2020-01-03,265.0
636,2020-01-03,265.0
993,2020-01-03,265.0
1281,2020-01-03,265.0


In [16]:
amazon_purchase=lst_df.drop_duplicates()

In [21]:
print(f'{amazon_purchase.shape[0]} number of times purchases were made in 2020')

165 number of times purchases were made in 2020


In [25]:
bill=amazon_purchase['amount'].sum(skipna=True,axis=0)
print(f'total amount spent was Rs.{bill:6.2f}')

total amount spent was Rs.106711.39
